## 本文件使用LSTM和GRU进行训练，做谣言检测任务

In [15]:
%matplotlib inline 
# 魔法命令，使用后画图不用show了

import numpy as np
import matplotlib.pyplot as plt
import re# 引入正则

import warnings
warnings.filterwarnings("ignore")

### 1.解压词向量并加载

#### 1.1解压词向量
本段仅第一次运行时需要执行，需要从 https://github.com/Embedding/Chinese-Word-Vectors 中的Pre-trained Chinese Word Vectors->Various Domains->Word2vec / Skip-Gram with Negative Sampling (SGNS)表格中找到对应微博的内容下载词向量

In [ ]:
import bz2# 用来解压文件

In [ ]:
with open("./embeddings/sgns.weibo.bigram", 'wb') as new_file, open("./embeddings/sgns.weibo.bigram.bz2", 'rb') as file:
    decompressor = bz2.BZ2Decompressor()
    for data in iter(lambda : file.read(100 * 1024), b''):
        new_file.write(decompressor.decompress(data))

#### 1.2加载词向量

In [16]:
from gensim.models import KeyedVectors# gensim用来加载预训练词向量

In [17]:
cn_model = KeyedVectors.load_word2vec_format('./embeddings/sgns.weibo.bigram', 
                                             binary=False,
                                             unicode_errors="ignore")

### 2.语料预处理

In [18]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

#### 2.1读取原始文本
* weibo：DataFrame存储的博文及其对应标签
* content：list存储的原始文本字符串
* label：标签，1为非谣言

In [19]:
import pandas as pd

In [20]:
weibo = pd.read_csv('./data/all_data.txt',sep='\t', names=['is_not_rumor','content'],encoding='utf-8')
weibo = weibo.dropna()#删除缺失值
weibo.head()

,is_not_rumor,content
0,0,【震惊：婴儿被害案另有隐情】周喜军儿子周磊为真凶，周磊平日游手好闲，江湖称号“经开三哥”。警...
1,1,郭德纲和于谦最新段子《屌丝青年》笑死人！！!http://t.cn/zlvEBLA 德云社庆...
2,1,【在喜欢的人面前反而害羞的男生】第一名 水瓶座（不敢正视喜欢的人）；第二名 金牛座（像小孩儿...
3,1,#21世纪明天日报# 【华为击中了小米什么？】先是和格力电器董事长董明珠立下10亿元的对赌协...
4,1,【曝林峰吴千语同居三月 年底厦门完婚】据港媒报道，林峰痴恋吴千语达爆灯程度，被网民笑如中了“...


In [21]:
weibo.shape

(3387, 2)

In [22]:
#将DataFrame中的Series转换为list
content = weibo.content.values.tolist()
label=weibo.is_not_rumor.values.tolist()

In [23]:
print (content[3:5])

['#21世纪明天日报# 【华为击中了小米什么？】先是和格力电器董事长董明珠立下10亿元的对赌协议，然后又在生日这天收到华为赤裸裸的战书，小米创始人雷军最近忙死了。华为“荣耀”来势汹汹，是否能够PK掉小米？', '【曝林峰吴千语同居三月 年底厦门完婚】据港媒报道，林峰痴恋吴千语达爆灯程度，被网民笑如中了“BB降”。近月，为多见吴千语几面，有豪宅不住，搬到吴千语香闺同居试婚。同居朝夕相对，二人更觉难舍难离，林峰终落实年底厦门摆酒迎娶吴千语。详情：http://t.cn/8shiAbn [吃惊]求真相！不过真心祝福！']


#### 2.2进行分词和tokenize
分词参考如下项目：
https://github.com/lancopku/PKUSeg-python

对每一条微博文本text：
1. 去掉每个样本的标点符号；
2. 用pkuseg分词，得到存放分词结果的cut_list；
3. 去掉cut_list中的停用词得到cut_list_clean；
3. 将分词结果cut_list_clean索引化（使用北京师范大学中文信息处理研究所与中国人民大学 DBIIR 实验室的研究者开源的"chinese-word-vectors"），这样每一例评价的文本变成一段索引数字，对应着预训练词向量模型中的词。

将每个text的结果存到train_tokens中。

In [24]:
import pkuseg

In [25]:
#导入停用词
stopwords=pd.read_csv("./stopwords/stopwords.txt",index_col=False,sep="\t",quoting=3,names=['stopword'], encoding='utf-8')
stopwords = stopwords.stopword.values.tolist()#转为list形式

In [26]:
seg = pkuseg.pkuseg(model_name='web')  # 程序会自动下载所对应的细领域模型

In [27]:
#这段是错的，经过处理的train_tokens除了不在字典中的词是0外，都没能换成词语对应的索引，效果在下个单元格的输出
train_tokens = []
for text in content:
    # 去掉标点
    text = re.sub("[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）]+", "",text)
    # pkuseg分词
    cut_list = seg.cut(text)

    #去除停用词
    cut_list_clean=[]
    for word in cut_list:
        if word in stopwords:
            continue
        cut_list_clean.append(word)
    
    #索引化
    for i, word in enumerate(cut_list_clean): # enumerate()
        try:
            # 将词转换为索引index
            #cut_list_clean[i] = cn_model.vocab[word].index
            cut_list[i] = cn_model.key_to_index [word]
        except KeyError:
            # 如果词不在字典中，则输出0
            cut_list_clean[i] = 0
    train_tokens.append(cut_list_clean)

In [28]:
for tokens in train_tokens:
    print(tokens)

['震惊', '婴儿', 0, '隐情', '周喜军', '儿子', '周磊', '真凶', '周磊', '平日', '游手好闲', '江湖', '称号', 0, '三哥', '警察', 0, 0, '指认', '现场', '发现', '可疑', '确认', '父亲', 0, '顶罪', '自首', '今日', '周磊', '归案', '婴儿', '父亲', '女友', 0, '劫车', '杀人', 0, '周磊', '周喜军', '盗车', '监控', '照片', '老子', '开车', '儿子', '开车']
['郭德纲', '于谦', '新', '段子', 0, '青年', '笑', '死人', 0, '德云', '社庆', 0, '郭德纲', '称', '三千', '城管', '收复', '日本']
['喜欢', '面前', '害羞', '男生', '第一', '名', '水瓶座', '正视', '喜欢', '名', '金牛座', '小孩儿', '脸红', 0, '名', '双子座', '躲开', '心仪', 0, '名', '巨蟹座', '悲观', '本性', '害羞', 0, '名', '狮子座', 0, '重放', '开']
[0, '明天', '日报', '华为', '击中', '小米', '先', '格力', '电器', '董事长', '董明珠', '立下', '10亿', '元', '对赌', '协议', '生日', '这天', '收到', '华为', '赤裸裸', '战书', '小米', '创始人', '雷军', 0, '华为', '荣耀', '来势汹汹', 'PK', '掉', '小米']
[0, '吴千语', '同居', '三月', '年底', '厦门', '完婚', 0, '报道', '林峰', '痴恋', '吴千语', '达', '爆灯', '程度', '网民', 0, '中', 0, 0, '吴千语', 0, '豪宅', '不住', '搬到', '吴千语', '香闺', '同居', '试婚', '同居', 0, 0, 0, 0, 0, '落实', '年底', '厦门', '摆酒', '迎娶', '吴千语', '详情', 0, '吃', 0, '真相', '真心', '祝福']
[0, '号', '休假', '蒙牛', 0, 0, '钢

In [31]:
#这段才是正确代码
train_tokens = []
for text in content:
    # 去掉标点
    text = re.sub("[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）]+", "",text)
    # pkuseg分词
    cut_list = seg.cut(text)

    #去除停用词
    cut_list_clean=[]
    for word in cut_list:
        if word in stopwords:
            continue
        cut_list_clean.append(word)
    
    #索引化
    for i, word in enumerate(cut_list_clean): # enumerate()
        try:
            # 将词转换为索引index
            #cut_list_clean[i] = cn_model.vocab[word].index
            cut_list_clean[i] = cn_model.key_to_index [word]
        except KeyError:
            # 如果词不在字典中，则输出0
            cut_list_clean[i] = 0
    train_tokens.append(cut_list_clean)

for tokens in train_tokens:
    print(tokens)

[6025, 4965, 0, 69894, 79402, 1130, 129471, 52951, 129471, 8826, 88955, 2769, 9285, 0, 13960, 2520, 0, 0, 83885, 449, 259, 24175, 2138, 2492, 0, 187390, 30972, 763, 129471, 130349, 4965, 2492, 4498, 0, 173707, 7542, 0, 129471, 79402, 116170, 8082, 231, 4424, 2122, 1130, 2122]
[6019, 19892, 217, 5131, 0, 924, 454, 8344, 0, 63253, 144923, 0, 6019, 754, 9423, 4073, 33788, 403]
[120, 2155, 10731, 3168, 153, 264, 5411, 17219, 120, 264, 5862, 18760, 13426, 0, 264, 5025, 28538, 9149, 0, 264, 2894, 11400, 13997, 10731, 0, 264, 4861, 0, 84065, 303]
[0, 337, 5066, 8427, 20082, 1334, 352, 43804, 9899, 4720, 73864, 43659, 14785, 185, 70565, 6520, 955, 7457, 793, 8427, 10777, 141497, 1334, 4420, 11875, 0, 8427, 8132, 48530, 4754, 1539, 1334]
[0, 172272, 21835, 2095, 4042, 472, 57713, 0, 1087, 60374, 118470, 172272, 796, 105829, 2063, 7452, 0, 71, 0, 0, 172272, 0, 12830, 3536, 11116, 172272, 141725, 21835, 127876, 21835, 0, 0, 0, 0, 0, 8734, 4042, 472, 86757, 40203, 172272, 1157, 0, 55, 0, 2411, 893

#### 2.3索引长度标准化
因为每段评语的长度是不一样的，如果单纯取最长的一个评语，并把其他评论填充为统一长度，会浪费太多资源，故取折中长度

In [32]:
# 获得所有tokens的长度
num_tokens = [len(tokens) for tokens in train_tokens]
num_tokens = np.array(num_tokens)
# 取tokens平均值并加上两个tokens的标准差，
# 假设tokens长度的分布为正态分布，则max_tokens这个值可以涵盖95%左右的样本
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

58

##### padding（填充）和truncating（修剪）
我们把文本转换为tokens（索引）之后，每一串索引的长度并不相等，所以为了方便模型的训练我们需要把索引的长度标准化，上面我们选择了max_tokens个可以涵盖95%训练样本的长度，接下来我们进行padding和truncating，我们一般采用'pre'的方法，这会在文本索引的前面填充0，因为根据一些研究资料中的实践，如果在文本索引后面填充0的话，会对模型造成一些不良影响。 

进行padding和truncating， 输入的train_tokens是一个list
返回的train_pad是一个numpy array

In [33]:
train_pad = pad_sequences(train_tokens, maxlen=max_tokens,
                            padding='pre', truncating='pre')

#### 2.4准备Embedding Matrix
* 作为模型的输入，需要准备一个维度为 (𝑛𝑢𝑚𝑤𝑜𝑟𝑑𝑠,𝑒𝑚𝑏𝑒𝑑𝑑𝑖𝑛𝑔𝑑𝑖𝑚) 的embedding矩阵，num words代表使用的词汇的数量。
* 使用预训练的词向量——[北京师范大学中文信息处理研究所与中国人民大学 DBIIR 实验室的研究者开源的"chinese-word-vectors"](https://github.com/Embedding/Chinese-Word-Vectors)；emdedding dimension在现在使用的预训练词向量模型中是300，每一个词汇都用一个长度为300的向量表示。
* 注意只选择使用前50k个使用频率最高的词，在这个预训练词向量模型中，一共有260万词汇量，如果全部使用在分类问题上会很浪费计算资源，因为训练样本很小，如果有更多的训练样本时，在分类问题上可以考虑减少使用的词汇量。

In [36]:
num_words = 50000
embedding_dim=300
# 初始化embedding_matrix，之后在keras上进行应用
embedding_matrix = np.zeros((num_words, embedding_dim))
# embedding_matrix为一个 [num_words，embedding_dim] 的矩阵
# 维度为 50000 * 300
for i in range(num_words):
    #embedding_matrix[i,:] = cn_model[cn_model.index2word[i]]#前50000个index对应的词的词向量
    embedding_matrix[i,:] = cn_model[cn_model.index_to_key[i]]
embedding_matrix = embedding_matrix.astype('float32')
# 检查index是否对应，
# 输出300意义为长度为300的embedding向量一一对应
#np.sum(cn_model[cn_model.index2word[333]] == embedding_matrix[333] )
np.sum(cn_model[cn_model.index_to_key[333]] == embedding_matrix[333] )

300

In [37]:
# 超出五万个词向量的词用0代替
train_pad[train_pad>=num_words ] = 0

# 准备target向量，前2000样本为1，后2000为0
train_target = np.array(label)

### 3.训练语料

In [38]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding, LSTM, Bidirectional
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau

#### 3.1划分训练集和测试集

In [39]:
# 进行训练和测试样本的分割
from sklearn.model_selection import train_test_split
# 90%的样本用来训练，剩余10%用来测试
X_train, X_test, y_train, y_test = train_test_split(train_pad,
                                                    train_target,
                                                    test_size=0.1,
                                                    random_state=12)

#### 3.2搭建网络结构
* 用keras搭建LSTM模型，第一层Embedding层将tokens索引转换为词向量矩阵，用神经网络处理文本（keras提供了embedding接口）
* 输入矩阵为(batchsize, maxtokens)，输出矩阵为(batchsize, maxtokens, embeddingdim)
* 将trainable设为False，使用预训练过的词向量

In [42]:
model = Sequential()
model.add(Embedding(num_words,
                    embedding_dim,
                    weights=[embedding_matrix],
                    input_length=max_tokens,
                    trainable=False))
model.add(Bidirectional(LSTM(units=64, return_sequences=True)))
model.add(Bidirectional(LSTM(units=32, return_sequences=False)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
optimizer=Adam(lr=1e-3)
#optimizer=Adam(learing_rate=1e-3)

==================model1==================

In [56]:
model1 = Sequential()
model1.add(Embedding(num_words,
                    embedding_dim,
                    weights=[embedding_matrix],
                    input_length=max_tokens,
                    trainable=False))
model1.add(Bidirectional(GRU(32)))
model1.add(Dense(6, activation='relu'))
model1.add(Dense(1, activation='sigmoid'))
optimizer=Adam(lr=1e-3)

#### 3.3模型配置

##### 模型保存（断点续训）、早停（early stoping）、学习率

In [45]:
import os

In [47]:
# 建立一个权重的存储点
checkpoint_save_path="./checkpoint/rumor_LSTM.ckpt"
print(checkpoint_save_path)
if os.path.exists(checkpoint_save_path+'.index'):
    print('----------load the model----------')
    model.load_weights(checkpoint_save_path)

./checkpoint/rumor_LSTM.ckpt


In [48]:
#保存参数和模型
checkpoint = ModelCheckpoint(filepath=checkpoint_save_path, monitor='val_loss',
                                      verbose=1, save_weights_only=True,
                                      save_best_only=True)

In [49]:
# 定义early stoping如果3个epoch内validation loss没有改善则停止训练
earlystopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

# 自动降低learning rate
lr_reduction = ReduceLROnPlateau(monitor='val_loss',
                                       factor=0.1, min_lr=1e-8, patience=0,
                                       verbose=1)
# 定义callback函数
callbacks = [
    earlystopping, 
#    checkpoint,
    lr_reduction
]

In [50]:
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

========================model1===================

In [57]:
# 建立一个权重的存储点
checkpoint_save_path1="./checkpoint/rumor_GRU.ckpt"
if os.path.exists(checkpoint_save_path1+'.index'):
    print('----------load the model----------')
    model1.load_weights(checkpoint_save_path1)

In [58]:
#保存参数和模型
checkpoint1 = ModelCheckpoint(filepath=checkpoint_save_path1, monitor='val_loss',
                                      verbose=1, save_weights_only=True,
                                      save_best_only=True)

In [59]:
model1.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

#### 3.4训练模型

In [54]:
model.fit(X_train, y_train,validation_split=0.1,epochs=20,batch_size=128,callbacks=callbacks)

Epoch 1/20
22/22 [==============================] - 10s 186ms/step - loss: 0.6297 - accuracy: 0.6478 - val_loss: 0.5133 - val_accuracy: 0.7672 - lr: 0.0010
Epoch 2/20
22/22 [==============================] - 3s 130ms/step - loss: 0.4689 - accuracy: 0.7783 - val_loss: 0.4434 - val_accuracy: 0.8066 - lr: 0.0010
Epoch 3/20
22/22 [==============================] - 3s 118ms/step - loss: 0.3972 - accuracy: 0.8250 - val_loss: 0.4190 - val_accuracy: 0.8098 - lr: 0.0010
Epoch 4/20
22/22 [==============================] - 3s 118ms/step - loss: 0.3545 - accuracy: 0.8487 - val_loss: 0.3791 - val_accuracy: 0.8459 - lr: 0.0010
Epoch 5/20
22/22 [==============================] - 3s 124ms/step - loss: 0.2805 - accuracy: 0.8870 - val_loss: 0.3591 - val_accuracy: 0.8295 - lr: 0.0010
Epoch 6/20
22/22 [==============================] - ETA: 0s - loss: 0.2363 - accuracy: 0.9103
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
22/22 [==============================] - 3s 126ms/ste

======================model1===========================

In [60]:
model1.fit(X_train, y_train,validation_split=0.1,epochs=20,batch_size=128,callbacks=callbacks)

Epoch 1/20
22/22 [==============================] - 4s 86ms/step - loss: 0.6869 - accuracy: 0.5385 - val_loss: 0.6740 - val_accuracy: 0.5475 - lr: 0.0010
Epoch 2/20
22/22 [==============================] - 1s 59ms/step - loss: 0.6619 - accuracy: 0.5797 - val_loss: 0.6524 - val_accuracy: 0.5639 - lr: 0.0010
Epoch 3/20
22/22 [==============================] - 1s 57ms/step - loss: 0.6242 - accuracy: 0.6274 - val_loss: 0.5941 - val_accuracy: 0.6328 - lr: 0.0010
Epoch 4/20
22/22 [==============================] - 1s 56ms/step - loss: 0.5448 - accuracy: 0.7601 - val_loss: 0.5322 - val_accuracy: 0.7738 - lr: 0.0010
Epoch 5/20
22/22 [==============================] - 1s 52ms/step - loss: 0.4862 - accuracy: 0.8144 - val_loss: 0.4989 - val_accuracy: 0.7934 - lr: 0.0010
Epoch 6/20
22/22 [==============================] - 1s 52ms/step - loss: 0.4472 - accuracy: 0.8432 - val_loss: 0.4153 - val_accuracy: 0.8262 - lr: 0.0010
Epoch 7/20
22/22 [==============================] - 1s 56ms/step - loss: 0.3

#### 3.5应用于测试集
首先对测试样本进行预测，得到了还算满意的准确度。之后我们定义一个预测函数，来预测输入的文本的极性，可见模型对于否定句和一些简单的逻辑结构都可以进行准确的判断。

In [61]:
result = model.evaluate(X_test, y_test)
print('Accuracy:{0:.2%}'.format(result[1]))

11/11 [==============================] - 0s 27ms/step - loss: 0.3976 - accuracy: 0.8319
Accuracy:83.19%


======================model1===========================

In [62]:
result = model1.evaluate(X_test, y_test)
print('Accuracy:{0:.2%}'.format(result[1]))

11/11 [==============================] - 0s 16ms/step - loss: 0.3525 - accuracy: 0.8525
Accuracy:85.25%


#### 3.6实例展示

In [65]:
def predict_rumor_LSTM(text,label):
    print(text)
    # 去标点
    text = re.sub("[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）]+", "",text)
    # 分词
    cut = seg.cut(text)
    #去除停用词
    cut_clean=[]
    for word in cut:
        if word in stopwords:
            continue
        cut_clean.append(word)
    # tokenize
    for i, word in enumerate(cut_clean):
        try:
            #cut_clean[i] = cn_model.vocab[word].index
            cut_clean[i] = cn_model.key_to_index[word]
            if cut_clean[i] >= 50000:
                cut_clean[i] = 0
        except KeyError:
            cut_clean[i] = 0
    # padding
    tokens_pad = pad_sequences([cut_clean], maxlen=max_tokens,
                           padding='pre', truncating='pre')
    # 预测
    dic={0:'谣言',1:'非谣言'}
    result = model.predict(x=tokens_pad)
    coef = result[0][0]
    if coef >= 0.5:
        print('真实是'+dic[label],'预测是非谣言','output=%.2f'%coef)
    else:
        print('真实是'+dic[label],'预测是谣言','output=%.2f'%coef)
    print('---------------------------------------------')

In [72]:
test_list = [
    '兴仁县今天抢小孩没抢走，把孩子母亲捅了一刀，看见这车的注意了，真事，车牌号辽HFM055！！！！！赶紧散播！ 都别带孩子出去瞎转悠了 尤其别让老人自己带孩子出去 太危险了 注意了！！！！辽HFM055北京现代朗动，在各学校门口抢小孩！！！110已经 证实！！全市通缉！！',
    '重庆真实新闻:2016年6月1日在重庆梁平县袁驿镇发生一起抢儿童事件，做案人三个中年男人，在三中学校到镇街上的一条小路上，把小孩直接弄晕(儿童是袁驿新幼儿园中班的一名学生)，正准备带走时被家长及时发现用棒子赶走了做案人，故此获救！请各位同胞们以此引起非常重视，希望大家有爱心的人传递下',
    '@尾熊C 要提前预习育儿知识的话，建议看一些小巫写的书，嘻嘻',
    '听说拜登要公开支持以色列进军加沙地带了',
    '我今天考试考了100分',
]
test_label=[0,0,1,0,1]
for i in range(len(test_list)):
    predict_rumor_LSTM(test_list[i],test_label[i])

兴仁县今天抢小孩没抢走，把孩子母亲捅了一刀，看见这车的注意了，真事，车牌号辽HFM055！！！！！赶紧散播！ 都别带孩子出去瞎转悠了 尤其别让老人自己带孩子出去 太危险了 注意了！！！！辽HFM055北京现代朗动，在各学校门口抢小孩！！！110已经 证实！！全市通缉！！
1/1 [==============================] - 0s 27ms/step
真实是谣言 预测是谣言 output=0.02
---------------------------------------------
重庆真实新闻:2016年6月1日在重庆梁平县袁驿镇发生一起抢儿童事件，做案人三个中年男人，在三中学校到镇街上的一条小路上，把小孩直接弄晕(儿童是袁驿新幼儿园中班的一名学生)，正准备带走时被家长及时发现用棒子赶走了做案人，故此获救！请各位同胞们以此引起非常重视，希望大家有爱心的人传递下
1/1 [==============================] - 0s 32ms/step
真实是谣言 预测是谣言 output=0.02
---------------------------------------------
@尾熊C 要提前预习育儿知识的话，建议看一些小巫写的书，嘻嘻
1/1 [==============================] - 0s 32ms/step
真实是非谣言 预测是非谣言 output=0.69
---------------------------------------------
听说拜登要公开支持以色列进军加沙地带了
1/1 [==============================] - 0s 29ms/step
真实是谣言 预测是非谣言 output=0.55
---------------------------------------------
我今天考试考了100分
1/1 [==============================] - 0s 29ms/step
真实是非谣言 预测是非谣言 output=0.57
---------------------------------------------


=====================model1=========================

In [ ]:
def predict_rumor_GRU(text,label):
    print(text)
    # 去标点
    text = re.sub("[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）]+", "",text)
    # 分词
    cut = seg.cut(text)
    #去除停用词
    cut_clean=[]
    for word in cut:
        if word in stopwords:
            continue
        cut_clean.append(word)
    # tokenize
    for i, word in enumerate(cut_clean):
        try:
            cut_clean[i] = cn_model.vocab[word].index
            if cut_clean[i] >= 50000:
                cut_clean[i] = 0
        except KeyError:
            cut_clean[i] = 0
    # padding
    tokens_pad = pad_sequences([cut_clean], maxlen=max_tokens,
                           padding='pre', truncating='pre')
    # 预测
    dic={0:'谣言',1:'非谣言'}
    result = model1.predict(x=tokens_pad)
    coef = result[0][0]
    if coef >= 0.5:
        print('真实是'+dic[label],'预测是非谣言','output=%.2f'%coef)
    else:
        print('真实是'+dic[label],'预测是谣言','output=%.2f'%coef)
    print('---------------------------------------------')

In [ ]:
test_list = [
    '兴仁县今天抢小孩没抢走，把孩子母亲捅了一刀，看见这车的注意了，真事，车牌号辽HFM055！！！！！赶紧散播！ 都别带孩子出去瞎转悠了 尤其别让老人自己带孩子出去 太危险了 注意了！！！！辽HFM055北京现代朗动，在各学校门口抢小孩！！！110已经 证实！！全市通缉！！',
    '重庆真实新闻:2016年6月1日在重庆梁平县袁驿镇发生一起抢儿童事件，做案人三个中年男人，在三中学校到镇街上的一条小路上，把小孩直接弄晕(儿童是袁驿新幼儿园中班的一名学生)，正准备带走时被家长及时发现用棒子赶走了做案人，故此获救！请各位同胞们以此引起非常重视，希望大家有爱心的人传递下',
    '@尾熊C 要提前预习育儿知识的话，建议看一些小巫写的书，嘻嘻',
]
test_label=[0,0,1]
for i in range(len(test_list)):
    predict_rumor_GRU(test_list[i],test_label[i])